In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# MUST MOUNT GOOGLE DRIVE FIRST OR THE CODE WILL NOT RUN
# These are all L2 data products, may add L1 to fill holes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

# This code takes local weather data from ncdc.noaa.gov and makes an output file and graphs certain values
# The values used are 'HOURLY', 'Celsius', 'CMAX', 'CMIN', 'SPEED', and 'GUST'

data = pd.read_csv('C:/Users/truet/emgt 630/605017983476dat.txt', sep='\s+',  # imports data and define na values
                   na_values=['0.00T', '*', '**', '***', '****', '*****', '******'])
name_change = {'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'MAX': 'FMAX',
               'MIN': 'FMIN'}  # convert names # of important data
data = data.rename(columns=name_change)  # renames the rows


def fahrToCelsius(fahrenheit):
    """
    Convert Fahrenheit into Celsius.

    Parameters
    ----------

    fahrenheit: float
        Input temperature in Fahrenheit (should be a number)
    """

    conversion = round((fahrenheit - 32) / 1.8, 2)  # Convert Fahrenheit to Celsius and rounds value
    return conversion


select_cols = ['TIME', 'SPEED', 'TEMP', 'GUST', 'FMAX', 'FMIN']  # Selecting specific data
data = data[select_cols]  # Defining the data set to the selected col
data['SPEED'] = round(data['SPEED'] * 0.44704, 2)  # converting mph to m/s
data['GUST'] = round(data['GUST'] * 0.44704, 2)
data['TIME_str'] = data['TIME'].astype(str)  # change time to a string
data['HOURLY'] = data['TIME_str'].str.slice(start=8, stop=10)
data['HOURLY'] = data['HOURLY'].astype(int)

col_name = 'Celsius'  # Create an empty column for the data
data[col_name] = None

for idx, row in data.iterrows():
    celsius = fahrToCelsius(row['TEMP'])  # Convert the Fahrenheit temperature of the row into Celsius
    data.loc[idx, col_name] = celsius  # Add that value into 'Celsius' column using the index of the row

col_name = 'CMAX'  # Create an empty column for the data
data[col_name] = None
for idx, row in data.iterrows():
    celsius = fahrToCelsius(row['FMAX'])  # Convert the Fahrenheit temperature of the row into Celsius
    data.loc[idx, col_name] = celsius  # Add that value into 'CMAX' column using the index of the row

col_name = 'CMIN'  # Create an empty column for the data
data[col_name] = None
for idx, row in data.iterrows():
    celsius = fahrToCelsius(row['FMIN'])  # Convert the Fahrenheit temperature of the row into Celsius
    data.loc[idx, col_name] = celsius  # Add that value into 'CMIN' column using the index of the row

important_cols = ['HOURLY', 'Celsius', 'CMAX', 'CMIN', 'SPEED',
                  'GUST']  # Defining columns which are used in the output
data = data[important_cols]
print("Below shows Hourly Surface Data")
print(data)  # Prints the data before writing
data.to_csv('Output.txt', float_format='%g', sep='\t')  # write the data to an output file

x = data['HOURLY']  # Defining variables to graph
y1 = data['Celsius']
y2 = data['CMAX']
y3 = data['CMIN']
y4 = data['SPEED']
y5 = data['GUST']

fig = plt.figure()  # figure for plot
ax1 = plt.subplot(211)  # first plot
plt.title('NCEI CDO Hourly Surface Data')
plt.xlabel('Time hourly', axes=ax1)  # Labled x axis
plt.ylabel('Temprature Celsius', axes=ax1)  # Labled y axis
ax1.margins(x=0)  # Defining margin
ax1.grid(True)  # Axis Grid
ax1.xaxis.set_major_locator(MultipleLocator(1))  # major x axis seprator
ax1.yaxis.set_major_locator(MultipleLocator(0.5))  # major y axis seprator
ax1.plot(x, y1, label='Hourly Temp')  # plots hourly with celsius
ax1.plot(x, y2, 'ro', label='Hourly max')  # plots hourly with celsius max
ax1.plot(x, y3, 'bs', label='Hourly min')  # plots hourly with celsius min
plt.legend()

ax2 = plt.subplot(212, sharex=ax1)  # second plot
plt.xlabel('Time hourly', axes=ax2)  # Labled x axis
plt.ylabel('Speed m/s', axes=ax2)  # Labled y axis
ax2.margins(x=0)  # Defining margin
ax2.grid(True)  # Axis Grid
ax2.xaxis.set_major_locator(MultipleLocator(1))  # major x axis seprator
ax2.yaxis.set_major_locator(MultipleLocator(0.5))  # major y axis seprator
ax2.plot(x, y4, label='WIND SPEED')  # plots hourly with speed
ax2.plot(x, y5, '#FFA500', label='GUST')  # plots hourly with gust
plt.legend()
plt.show()
